In [1]:
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [2]:
API_KEY = "REPLACE-ME"

In [51]:
from openai import OpenAI
from pydantic import BaseModel
from IPython.display import Markdown

client = OpenAI(api_key=API_KEY)


class EvaluationResponse(BaseModel):
    assessment: str
    score: float
    feedback: str

def show_result(result):
    print("ASSESSMENT")
    display(Markdown(result.assessment))

    print()
    print("FEEDBACK")
    display(Markdown(result.feedback))

    print()
    print("SCORE")
    display(Markdown(str(result.score)))


def read_txt_file(filename):
  try:
    with open(filename, 'r') as file:
      content = file.read()
    return content
  except FileNotFoundError:
    print(f"File not found: {filename}")
    return None


def generate_messages(essay_text, reference_answer):
    system_prompt = f"""
        Kamu adalah guru AI yang bertugas untuk mengecek dan menilai jawaban esai murid-murid.
        Tugasmu adalah memberikan penilaian yang objektif berdasarkan kesesuaian jawaban murid dengan soal yang diberikan.
        Berikut ini adalah beberapa aturan yang harus kamu ikuti saat memberikan penilaian:
        1. Relevansi Konten: Pastikan jawaban murid sesuai dengan pertanyaan yang diajukan. Jawaban yang sangat relevan dengan soal harus mendapatkan nilai tinggi, sedangkan jawaban yang tidak relevan harus mendapatkan nilai rendah.
        2. Kedalaman Argumen: Evaluasi apakah murid memberikan penjelasan yang mendalam dan argumentasi yang kuat. Jawaban yang mencakup detail penting dan menunjukkan pemahaman mendalam harus mendapat nilai tinggi.
        3. Kejelasan dan Ketepatan: Nilai jawaban berdasarkan seberapa jelas dan tepat informasi yang disampaikan. Jawaban yang jelas, terstruktur dengan baik, dan bebas dari kesalahan konseptual harus mendapat nilai tinggi.
        4. Penggunaan Contoh: Pertimbangkan apakah murid menggunakan contoh yang relevan untuk mendukung argumennya. Penggunaan contoh yang tepat akan meningkatkan nilai.
        5. Tata Bahasa dan Penulisan: Meskipun tidak menjadi kriteria utama, jawaban yang bebas dari kesalahan tata bahasa dan penulisan akan mendapat poin tambahan.
        6. Skala Penilaian: Berikan nilai pada skala 0 - 10. Nilai 0 diberikan jika jawaban tidak sesuai sama sekali dengan soal, sedangkan nilai 10 diberikan untuk jawaban yang sangat sesuai, mendalam, dan jelas.

        Setelah mengevaluasi jawaban berdasarkan kriteria di atas, berikan penilaian setiap jawaban, nilai akhir, dan berikan sedikit feedback atau alasan mengapa jawaban tersebut mendapatkan nilai tersebut.

        *assessment*:
        ini berisi penilaian dari semua jawaban murid dengan format sebagai berikut:
        **Soal 1**: Nilai: (0 - 10)/10. feedback singkat mengenai apa yang baik dan apa yang bisa ditingkatkan dalam jawaban ini.\n
        **Soal 2**: Nilai: (0 - 10)/10. feedback singkat mengenai apa yang baik dan apa yang bisa ditingkatkan dalam jawaban ini.\n
        Contoh assessment:
        **Soal 1**: Nilai: 6/10. Jawaban menjelaskan konsep dasar OOP dengan singkat, tetapi tidak mencakup semua konsep penting seperti polymorphism dan abstraction. Nilai: 6/10 \n
        **Soal 2**: Nilai: 6/10. Jawaban cukup mendasar dan kurang mendalam, hanya menyebutkan pentingnya tanpa contoh konkret. Nilai: 6/10 \n
        **Soal 3**: Nilai: 5/10. Penjelasan sangat singkat dan kurang detail tentang kapan masing-masing paradigma lebih sesuai digunakan. Nilai: 5/10 \n

        *score*
        ini berisi nilai akhir dari semua jawaban murid dari 0 - 10 berupa float
        Contoh score:
        5.6

        *feedback*
        ini berisi sedikit ringkasan feedback dari semua jawaban murid
        Contoh feedback:
        Jawaban ini menunjukkan pemahaman yang solid tentang berbagai konsep pemrograman. Meskipun ada beberapa penjelasan yang bisa diperdalam, umumnya jawaban sangat relevan dengan pertanyaan dan menggunakan bahasa yang jelas dan mudah dipahami. Beberapa bagian bisa lebih terperinci untuk mengangkat kualitas jawaban.

        Gunakan bahasa indonesia

        Ini adalah soal beserta jawaban yang bisa kamu gunakan sebagai pengetahuan kamu dan referensi untuk kamu memberikan penilaian:
        {reference_answer}

        Nantinya saya akan memberikan jawaban murid ke kamu
    """
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": essay_text},
    ]


def evaluate_essay(essay_text, reference_answer):
    try:
        messages = generate_messages(essay_text, reference_answer)
        response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=messages,
            response_format=EvaluationResponse,
        )

        return response.choices[0].message.parsed
    except Exception as exception:
        print(f"Error {type(exception)} occured: {exception}")
        raise exception


In [52]:
reference_answer = read_txt_file("/content/soal dan jawaban.txt")

In [53]:
print("Student 1")
print()
student_answer = read_txt_file("/content/jawaban1.txt")
show_result(evaluate_essay(student_answer, reference_answer))

Student 1

ASSESSMENT


**Soal 1**: Nilai: 8/10. Jawaban cukup baik menjelaskan konsep dasar OOP dan membandingkannya dengan pemrograman prosedural, tetapi kurang mendalam dalam penjelasan mengenai polymorphism dan abstraction. Contoh yang diberikan juga kurang mencolok. 

**Soal 2**: Nilai: 8/10. Penjelasan mengenai pentingnya algoritma dan struktur data sangat baik dan langsung to the point, dengan contoh yang relevan. Meski begitu, bisa digali lebih dalam mengenai dampak pilihan struktur data. 

**Soal 3**: Nilai: 7/10. Menjelaskan dengan baik perbedaan antara pemrograman fungsional dan OOP, namun bisa lebih mendalam dalam menekankan skenario di mana masing-masing paradigma akan lebih diutamakan. 

**Soal 4**: Nilai: 9/10. Jawaban menjelaskan dengan jelas pentingnya error handling dan memberikan contoh yang relevan. Penjelasan debugging juga cukup solid. Hanya sedikit ruang untuk meningkatkan kedalaman. 

**Soal 5**: Nilai: 8/10. Penjelasan mengenai arsitektur aplikasi web dan peran frontend serta backend cukup jelas dan terstruktur baik. Akan lebih baik jika ada penjelasan tambahan mengenai contoh interaksi API. 

**Soal 6**: Nilai: 7/10. Menyampaikan pengertian tentang pemrograman paralel dan multithreading dengan cukup baik, namun kurang memberikan contoh konkret, serta tantangan yang lebih jelas dalam implementasinya. 

**Soal 7**: Nilai: 8/10. Jawaban baik dalam menjelaskan pentingnya keamanan dalam pemrograman, lengkap dengan beberapa praktik baik, namun mungkin bisa memperdalam menjelaskan tipe-tipe serangan yang lebih spesifik. 

**Soal 8**: Nilai: 8/10. Penjelasan mengenai VCS dan Git sudah baik dan cukup lengkap, walaupun bisa disoroti lebih banyak fungsi atau fitur Git yang mendukung kolaborasi. 

**Soal 9**: Nilai: 9/10. Jawaban cukup kuat pada peran pemrograman dalam machine learning dengan contoh pustaka Python yang spesifik. Penjelasannya cukup mendalam dan jelas. 

**Soal 10**: Nilai: 8/10. Menjelaskan dengan baik tentang API dan tantangan dalam mendesainnya. Namun, bisa lebih mendalam dalam contoh spesifik dari masalah yang mungkin dihadapi. 




FEEDBACK


Secara keseluruhan, jawaban ini menunjukkan pemahaman yang solid tentang berbagai aspek pemrograman yang dipertanyakan. Meskipun ada beberapa bagian yang bisa lebih mendalam atau terperinci, jawaban yang diberikan tetap relevan dan jelas serta mengikuti struktur yang baik. Penjelasan yang diberikan umumnya mudah dimengerti dan bermanfaat, tetapi penambahan contoh konkret dan elaborasi pada beberapa konsep dapat meningkatkan kualitas jawaban.


SCORE


8.1

In [54]:
print("Student 2")
print()
student_answer = read_txt_file("/content/jawaban2.txt")
show_result(evaluate_essay(student_answer, reference_answer))

Student 2

ASSESSMENT


**Soal 1**: Nilai: 5/10. Jawaban menjelaskan beberapa konsep dasar OOP seperti objek, inheritance, dan encapsulation, namun kurang mendalam dan tidak memberikan perbandingan yang jelas dengan paradigma prosedural. Contoh yang diberikan juga minim sehingga tidak cukup untuk memberi gambaran yang komprehensif.

**Soal 2**: Nilai: 6/10. Jawaban menyatakan pentingnya algoritma dan struktur data, tetapi tidak memberikan contoh konkret atau mendalam yang menunjukkan bagaimana mereka mempengaruhi efisiensi program.

**Soal 3**: Nilai: 6/10. Penjelasan mengenai perbedaan antara pemrograman fungsional dan OOP cukup jelas, namun kurang mendalam dalam menggambarkan kapan dan mengapa salah satu paradigma lebih baik dari yang lain.

**Soal 4**: Nilai: 6/10. Jawaban mencakup definisi dasar tentang error handling dan debugging, tetapi argumentasi bisa lebih dalam dengan contoh spesifik yang menunjukkan aplikasi konsep tersebut.

**Soal 5**: Nilai: 7/10. Menjelaskan esensi frontend dan backend dengan baik, tetapi masih kurang dalam memberikan contoh spesifik atau menjelaskan bagaimana API berfungsi dalam konteks arsitektur web.

**Soal 6**: Nilai: 6/10. Menyebutkan pemrograman paralel dan multithreading dengan benar, namun tidak memberikan penjelasan lebih lanjut tentang keuntungannya atau tantangannya.

**Soal 7**: Nilai: 5/10. Menyentuh pentingnya keamanan, tetapi jawaban ini terlalu singkat dan tidak membahas praktik terbaik dengan detail.

**Soal 8**: Nilai: 6/10. Menjelaskan fungsi VCS dan contoh Git cukup baik, tetapi tidak memberikan detail lebih lanjut tentang kelebihan Git dalam kolaborasi atau manajemen versi.

**Soal 9**: Nilai: 6/10. Menyatakan aspek dasar pemrograman dalam machine learning dengan Python, tetapi kurang menjelaskan peran spesifik dari pustaka Python dalam konteks itu.

**Soal 10**: Nilai: 6/10. Memberikan pengertian dasar tentang API, namun kurang rincian tentang tantangan dalam perancangan dan implementasi.



FEEDBACK


Jawaban menunjukkan pemahaman dasar tentang konsep-konsep penting dalam pemrograman, namun banyak jawaban yang terasa cukup dangkal dan kurang mendalam. Meskipun ada penjelasan yang benar, kurangnya contoh konkret dan detail membuat banyak bagian tidak cukup mengesankan. Perlu pengembangan lebih lanjut terutama dalam menjelaskan konsep dan menjawab dengan lebih lengkap.


SCORE


6.0

In [55]:
print("Student 3")
print()
student_answer = read_txt_file("/content/jawaban3.txt")
show_result(evaluate_essay(student_answer, reference_answer))

Student 3

ASSESSMENT


**Soal 1**: Nilai: 9/10. Jawaban sangat baik dan mencakup semua konsep utama OOP secara jelas. Penjelasan mengenai perbedaan dengan prosedural juga baik, tetapi contoh kasus bisa lebih spesifik untuk nilai lebih tinggi.

**Soal 2**: Nilai: 8/10. Penjelasan tentang pentingnya algoritma dan struktur data sangat baik, ditunjang contoh yang relevan. Namun, penjabaran tentang efisiensi bisa lebih dilengkapi dengan lebih banyak contoh.

**Soal 3**: Nilai: 8/10. Jawaban menyajikan perbedaan dengan jelas dan memberikan konteks penggunaan masing-masing paradigma. Meskipun ada penjelasan yang baik, penambahan contoh konkret akan meningkatkan kualitas jawaban.

**Soal 4**: Nilai: 8/10. Penjelasan tentang penanganan kesalahan sangat tepat, berisi contoh jelas. Namun, bisa lebih diperluas dengan menjelaskan lebih banyak teknik debugging yang berbeda.

**Soal 5**: Nilai: 9/10. Jawaban menjelaskan arsitektur modern dengan baik dan detail interaksi antara frontend dan backend. Penjelasan singkat bisa ditambahkan untuk menambah kedalaman.

**Soal 6**: Nilai: 8/10. Penjelasan tentang pemrograman paralel dan multithreading baik, tetapi bisa lebih lengkap dengan contoh kasus nyata di mana keduanya diterapkan.

**Soal 7**: Nilai: 9/10. Jawaban menyentuh semua titik penting mengenai keamanan dan praktik terbaiknya dengan baik dan jelas. Mungkin bisa disertakan contoh serangan agar lebih relevan.

**Soal 8**: Nilai: 9/10. Jawaban menjelaskan pentingnya VCS dan penggunaan Git dengan jelas dan terstruktur. Mencakup banyak aspek penting tetapi bisa lebih mendalam tentang beberapa fitur Git yang lebih avancé.

**Soal 9**: Nilai: 9/10. Penjelasan sangat baik dan terperinci, serta menyoroti peran Python dengan bagus. Contoh penggunaan pustaka dapat ditambahkan untuk mendemonstrasikan aplikasinya lebih lanjut.

**Soal 10**: Nilai: 8/10. Jawaban memberikan pemahaman yang tepat tentang API meskipun tantangan dalam desain bisa lebih diperjelas dengan contoh konkret. 



FEEDBACK


Jawaban ini menunjukkan pemahaman yang solid tentang berbagai konsep pemrograman. Meskipun ada beberapa bagian yang bisa lebih mendalam atau disertai contoh konkret, sebagian besar jawaban sangat relevan dengan pertanyaan dan disampaikan dengan bahasa yang jelas dan terstruktur.


SCORE


8.5